# 4. CNN-LSTM

In [1]:
import numpy as np
import pandas as pd
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, ReLU, LSTM
from tensorflow.keras.layers import TimeDistributed
from focal_loss import BinaryFocalLoss

from sklearn.metrics import roc_curve

2023-10-03 19:25:15.229056: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-10-03 19:25:16.741430: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 19:25:16.745730: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-10-03 19:25:16.771230: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: NVIDIA TITAN Xp computeCapability: 6.1
coreClock: 1.582GHz coreCount: 30 deviceMemorySize: 11.90GiB deviceMemoryBandwidth: 510.07GiB/s
2023-10-03 19:25:16.771263: I tensorflow/stream_executor/platform/default/dso_loader.cc:5

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5423443410737273241
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11974344704
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12249524192495599302
physical_device_desc: "device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:05:00.0, compute capability: 6.1"
]


In [2]:
feature = ['flux',
           'zero_month',
           'zero_week',
           'continuous_zero',
           'diff_week_day',
           'diff_month',
           'std_week',
           'diff_std',
           'rec_day',
           'rec_week',
           'rec_2week',
           'rec_month',
           'rec_day2',
           'rec_week2',
           'rec_2week2',
           'rec_month2', ]

total_df = pd.read_csv("../final_data.csv")
total_data = total_df.dropna().loc[:, ['label'] + feature].to_numpy()

window_n = 5
stride = 12
window_size = 24

normal_data = []
abnormal_data = []
where_abnormal = np.argwhere(total_data[:,0]==1)

# normal_data 추출 과정
for i in range(total_data.shape[0] - (window_size + (window_n - 1) * stride)):
  # 무작위성 부여
  if np.random.rand() > 0.30:
    continue
  
  sliced = []
  for w_n in range(window_n):
    # sliced 에 window_n번 window_size 행만큼 자른 데이터 append
    # sliced 에 5번 24x17 추가
    sliced.append(total_data[(w_n * stride) + i : (w_n * stride) + i + window_size])
  
  sliced = np.array(sliced)

  # 자른 데이터 label이 모두 정상이면 normal 추가
  if np.all(sliced[:,:,0] == 0):
    normal_data.append(sliced[:, :, 1:])

normal_data = np.array(normal_data).astype(np.float32)



# abnormal_data 추출 과정
for ab in where_abnormal:
    # 이전 24 + 12*4 시간의 data를 보고 진단 == 72시간
    # 그냥 ab 로 하면 list --> ab[0]
    i = ab[0] + 1 - window_size - stride * (window_n - 1)
    if (i < 0):
        continue
    
    sliced = []
    for w_n in range(window_n):
        sliced.append(total_data[i + (w_n * stride) : i + (w_n * stride) + window_size, 1:])
    
    sliced = np.array(sliced)
    abnormal_data.append(sliced)

#abnormal_data : 고장 직전 72시간의 데이터를 포함
abnormal_data = np.array(abnormal_data).astype(np.float32)

# Index Shuffle
random_idx = np.arange(normal_data.shape[0])
np.random.shuffle(random_idx)
normal_data = normal_data[random_idx]

random_idx = np.arange(abnormal_data.shape[0])
np.random.shuffle(random_idx)
abnormal_data = abnormal_data[random_idx]


cut = normal_data.shape[0]//10

# Training Dataset
training_x = normal_data[:cut]
training_y = [[1, 0] for _ in range(cut)]

# abnormal data oversampling 200 times
duplicate_weight = 200
for _ in range(duplicate_weight):
  training_x = np.concatenate([training_x, abnormal_data[:abnormal_data.shape[0]//2]], axis = 0)


training_y = training_y + [[0, 1] for _ in range(training_x.shape[0]-len(training_y))]
training_y = np.array(training_y)


# Test Dataset
test_x = normal_data[cut:cut*2]
test_y = [[1, 0] for _ in range(cut)]

test_x = np.concatenate([test_x, abnormal_data[abnormal_data.shape[0]//2:]], axis = 0)
test_y = test_y + [[0, 1] for _ in range(test_x.shape[0]-len(test_y))]

test_y = np.array(test_y)


# Dataset Labeling
# normal : [1,0]
# abnormal : [0,1]

print(training_x.shape)
print(training_y.shape)
print(test_x.shape)
print(test_y.shape)

print("training data 중 abnormal : ", sum(training_y[:,1]))

(183311, 5, 24, 16)
(183311, 2)
(86000, 5, 24, 16)
(86000, 2)
training data 중 abnormal :  97800


- Cross Entropy Loss

In [5]:
l2param=0.001

model = Sequential()
model.add(TimeDistributed(
  Conv1D(32, 4, padding = "valid",
         kernel_regularizer=tf.keras.regularizers.l2(l2param))
  ))
model.add(TimeDistributed(ReLU()))
model.add(TimeDistributed(MaxPooling1D(2)))
model.add(TimeDistributed(Flatten()))

model.add(LSTM(units=256,
              kernel_regularizer=tf.keras.regularizers.l2(l2param)
              ))
model.add(Dense(2, activation='sigmoid',
                kernel_regularizer=tf.keras.regularizers.l2(l2param)))

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0005), metrics=['accuracy'])

start = time.time()
history = model.fit(training_x, training_y, epochs=10, batch_size=256, validation_split=0.1)
end = time.time()

pred = model.predict(test_x, batch_size=256)[:,1]
test_ground_truth = test_y[:,1]
fpr, tpr, thresholds = roc_curve(test_ground_truth, pred)

# test_y
# normal : [1,0]
# abnormal : [0,1]

for i in range(len(fpr)):
        if fpr[i] > 0.001: #0.1%
            # fpr 이 thresholds 값 이상이 될 때 그 index 를 찾음
            if (i > 0): i -= 1
            break

predicted = pred >= thresholds[i]
TP = int(sum(np.logical_and(test_ground_truth==1,  predicted==1)))
TN = int(sum(np.logical_and(test_ground_truth==0,  predicted==0)))
FP = int(sum(np.logical_and(test_ground_truth==0,  predicted==1)))
FN = int(sum(np.logical_and(test_ground_truth==1,  predicted==0)))

Precision = TP/(TP+FP)
Recall = TP/(TP+FN)

print(TP, TN, FP, FN)

print("precision: ", Precision)
print("recall: ", Recall)
print("F-measure:", (2*Precision*Recall)/(Precision+Recall+1e-7))
print("MCC:", (TP*TN-FP*FN)/(np.sqrt(float((TP+FN)*(TP+FP)*(TN+FP)*(TN+FN)))+1e-7))
print("train time : ", end-start)

Epoch 1/10
645/645 [==============================] - 7s 7ms/step - loss: 0.3860 - accuracy: 0.8749 - val_loss: 0.3866 - val_accuracy: 0.7426
Epoch 2/10
645/645 [==============================] - 4s 6ms/step - loss: 0.2814 - accuracy: 0.8888 - val_loss: 0.2767 - val_accuracy: 0.9225
Epoch 3/10
645/645 [==============================] - 4s 6ms/step - loss: 0.2496 - accuracy: 0.9132 - val_loss: 0.2640 - val_accuracy: 0.9040
Epoch 4/10
645/645 [==============================] - 4s 6ms/step - loss: 0.2142 - accuracy: 0.9328 - val_loss: 0.2321 - val_accuracy: 0.8938
Epoch 5/10
645/645 [==============================] - 4s 6ms/step - loss: 0.1887 - accuracy: 0.9448 - val_loss: 0.1766 - val_accuracy: 0.9594
Epoch 6/10
645/645 [==============================] - 4s 6ms/step - loss: 0.1644 - accuracy: 0.9534 - val_loss: 0.1667 - val_accuracy: 0.9614
Epoch 7/10
645/645 [==============================] - 4s 6ms/step - loss: 0.1540 - accuracy: 0.9551 - val_loss: 0.1742 - val_accuracy: 0.9492
Epoch 

- Focal Loss

In [6]:
l2param=0.001

gamm = [0.5, 1, 1.5, 2.0]
for gam in gamm :
    
    model = Sequential()
    model.add(TimeDistributed(
      Conv1D(32, 4, padding = "valid",
            kernel_regularizer=tf.keras.regularizers.l2(l2param))
      ))
    model.add(TimeDistributed(ReLU()))
    model.add(TimeDistributed(MaxPooling1D(2)))
    model.add(TimeDistributed(Flatten()))

    model.add(LSTM(units=256,
                  kernel_regularizer=tf.keras.regularizers.l2(l2param)
                  ))
    model.add(Dense(2, activation='sigmoid',
                    kernel_regularizer=tf.keras.regularizers.l2(l2param)))
    model.compile(loss=BinaryFocalLoss(gamma=gam), optimizer=Adam(learning_rate=0.0005), metrics=['accuracy'])
    model.fit(training_x, training_y, epochs=10, batch_size=256, verbose=0, validation_split=0.1)

    print('gamma = {}'.format(gam))

    pred = model.predict(test_x, batch_size=256)[:,1]
    test_ground_truth = test_y[:,1]
    fpr, tpr, thresholds = roc_curve(test_ground_truth, pred)

    # test_y
    # normal : [1,0]
    # abnormal : [0,1]

    for i in range(len(fpr)):
            if fpr[i] > 0.001: #0.1%
                # fpr 이 thresholds 값 이상이 될 때 그 index 를 찾음
                if (i > 0): i -= 1
                break

    predicted = pred >= thresholds[i]
    TP = int(sum(np.logical_and(test_ground_truth==1,  predicted==1)))
    TN = int(sum(np.logical_and(test_ground_truth==0,  predicted==0)))
    FP = int(sum(np.logical_and(test_ground_truth==0,  predicted==1)))
    FN = int(sum(np.logical_and(test_ground_truth==1,  predicted==0)))

    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)

    print(TP, TN, FP, FN)

    print("precision: ", Precision)
    print("recall: ", Recall)
    print("F-measure:", (2*Precision*Recall)/(Precision+Recall+1e-7))
    print("MCC:", (TP*TN-FP*FN)/(np.sqrt(float((TP+FN)*(TP+FP)*(TN+FP)*(TN+FN)))+1e-7))
    print('='*20)

gamma = 0.1
407 85348 82 82
precision:  0.8323108384458078
recall:  0.8323108384458078
F-measure: 0.8323107884458106
MCC: 0.8313509882760762
gamma = 0.2
425 85347 83 64
precision:  0.8366141732283464
recall:  0.869120654396728
F-measure: 0.8525576230372189
MCC: 0.851854048959945
gamma = 0.30000000000000004
414 85357 73 75
precision:  0.8501026694045175
recall:  0.8466257668711656
F-measure: 0.8483606057379178
MCC: 0.8474962630502637
gamma = 0.4
426 85360 70 63
precision:  0.8588709677419355
recall:  0.8711656441717791
F-measure: 0.8649745692918682
MCC: 0.8642182876166802
gamma = 0.5
424 85345 85 65
precision:  0.8330058939096268
recall:  0.8670756646216768
F-measure: 0.8496993488176784
MCC: 0.848994233366785
gamma = 0.6
406 85347 83 83
precision:  0.8302658486707567
recall:  0.8302658486707567
F-measure: 0.8302657986707597
MCC: 0.8292942930111503
gamma = 0.7000000000000001
422 85349 81 67
precision:  0.8389662027833003
recall:  0.8629856850715747
F-measure: 0.8508064016228649
MCC: 0.85